In [7]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from numpy import sqrt
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [1]:
ml=pd.read_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Machine learning\ML plasma.xlsx", header=0, index_col="Gene names")
ml.head()

,Control 1,Control 2,Control 3,Control 4,Control 5,Control 6,Control 7,Control 8,Control 9,Control 10,...,ALS 20,ALS 21,ALS 22,ALS 23,ALS 24,ALS 25,ALS 26,ALS 27,ALS 28,ALS 29
Gene names,,,,,,,,,,,,,,,,,,,,,
A1BG,26.95427,26.04318,26.86112,26.84817,26.46922,26.64471,26.84086,26.65403,27.17303,26.97836,...,26.45931,27.16346,26.90058,26.45042,26.73480,26.83473,27.07398,26.92603,26.98934,26.40678
A2M,34.05310,33.97562,34.44289,34.01380,33.76643,33.97143,33.94585,33.94951,34.24133,34.00864,...,33.93526,33.91474,33.47206,34.03625,33.86673,33.92804,33.91019,33.32133,34.02383,33.80540
ACTB,30.33841,29.20109,30.07712,30.10600,29.82487,29.17995,30.12866,29.74278,29.49278,29.82696,...,29.40878,30.39734,30.23123,29.45146,30.39703,30.15943,30.55499,30.32978,30.44391,29.81420
ADIPOQ,29.63951,28.92731,30.02842,29.37902,30.37171,31.21824,29.76444,28.42606,29.57760,30.02842,...,31.13767,30.18139,30.42482,29.37968,29.46410,30.76462,29.90928,28.78578,30.34604,31.70989
AFM,28.50805,28.99558,30.35100,28.39239,23.28411,28.72576,27.65581,28.57423,27.04895,26.97367,...,28.43782,27.65103,28.80713,28.12736,27.38154,27.27897,26.52943,28.17736,27.89406,29.41353


In [2]:
ml.shape

(198, 57)

In [3]:
#Columns belonging to Group 1 and 2
ALS_columns = [col for col in ml.columns if "ALS" in col]
Control_columns = [col for col in ml.columns if "Control" in col]


# Filter rows where at least 50% of Group 1 values are present, do the same for Group 2
filtered_als = ml[ml[ALS_columns].notna().mean(axis=1) >= 0.5]
cont_filt = filtered_als[filtered_als[Control_columns].notna().mean(axis=1) >= 0.5]

In [4]:
cont_filt.shape

(198, 57)

In [5]:
# Automatically identify group columns by their prefixes
group1= cont_filt.loc[:, cont_filt.columns.str.startswith('Control')]
group2= cont_filt.loc[:, cont_filt.columns.str.startswith('ALS ')]

In [8]:
#stat, p_value = mannwhitneyu(group1.iloc[0], group2.iloc[0], alternative='two-sided')

stats = []
p_vals = []
for i in range(group1.shape[0]):
    stat, p_value = mannwhitneyu(group1.iloc[i], group2.iloc[i], alternative='two-sided', method ="exact")
    stats.append(stat)
    p_vals.append(p_value)
    

In [11]:
# Add the results DataFrame to the original data DataFrame
cont_filt['U-stat'] = stats
cont_filt['p_value'] = p_vals
cont_filt.head()

,Control 1,Control 2,Control 3,Control 4,Control 5,Control 6,Control 7,Control 8,Control 9,Control 10,...,ALS 22,ALS 23,ALS 24,ALS 25,ALS 26,ALS 27,ALS 28,ALS 29,U-stat,p_value
Gene names,,,,,,,,,,,,,,,,,,,,,
A1BG,26.95427,26.04318,26.86112,26.84817,26.46922,26.64471,26.84086,26.65403,27.17303,26.97836,...,26.90058,26.45042,26.73480,26.83473,27.07398,26.92603,26.98934,26.40678,455.0,0.441766
A2M,34.05310,33.97562,34.44289,34.01380,33.76643,33.97143,33.94585,33.94951,34.24133,34.00864,...,33.47206,34.03625,33.86673,33.92804,33.91019,33.32133,34.02383,33.80540,582.0,0.004503
ACTB,30.33841,29.20109,30.07712,30.10600,29.82487,29.17995,30.12866,29.74278,29.49278,29.82696,...,30.23123,29.45146,30.39703,30.15943,30.55499,30.32978,30.44391,29.81420,362.0,0.490382
ADIPOQ,29.63951,28.92731,30.02842,29.37902,30.37171,31.21824,29.76444,28.42606,29.57760,30.02842,...,30.42482,29.37968,29.46410,30.76462,29.90928,28.78578,30.34604,31.70989,399.0,0.918017
AFM,28.50805,28.99558,30.35100,28.39239,23.28411,28.72576,27.65581,28.57423,27.04895,26.97367,...,28.80713,28.12736,27.38154,27.27897,26.52943,28.17736,27.89406,29.41353,460.5,0.395990


In [13]:
# Apply Benjamini-Hochberg correction (optional)
_, p_adjusted, _, _ = multipletests(cont_filt['p_value'], method='fdr_bh')
cont_filt['p_adjusted_bh'] = p_adjusted
cont_filt.head()

,Control 1,Control 2,Control 3,Control 4,Control 5,Control 6,Control 7,Control 8,Control 9,Control 10,...,ALS 24,ALS 25,ALS 26,ALS 27,ALS 28,ALS 29,U-stat,p_value,p_adjusted,p_adjusted_bh
Gene names,,,,,,,,,,,,,,,,,,,,,
A1BG,26.95427,26.04318,26.86112,26.84817,26.46922,26.64471,26.84086,26.65403,27.17303,26.97836,...,26.73480,26.83473,27.07398,26.92603,26.98934,26.40678,455.0,0.441766,0.920734,0.920734
A2M,34.05310,33.97562,34.44289,34.01380,33.76643,33.97143,33.94585,33.94951,34.24133,34.00864,...,33.86673,33.92804,33.91019,33.32133,34.02383,33.80540,582.0,0.004503,0.297230,0.297230
ACTB,30.33841,29.20109,30.07712,30.10600,29.82487,29.17995,30.12866,29.74278,29.49278,29.82696,...,30.39703,30.15943,30.55499,30.32978,30.44391,29.81420,362.0,0.490382,0.964701,0.964701
ADIPOQ,29.63951,28.92731,30.02842,29.37902,30.37171,31.21824,29.76444,28.42606,29.57760,30.02842,...,29.46410,30.76462,29.90928,28.78578,30.34604,31.70989,399.0,0.918017,0.964701,0.964701
AFM,28.50805,28.99558,30.35100,28.39239,23.28411,28.72576,27.65581,28.57423,27.04895,26.97367,...,27.38154,27.27897,26.52943,28.17736,27.89406,29.41353,460.5,0.395990,0.900817,0.900817


In [14]:
_, p_adjusted, _, _ = multipletests(cont_filt['p_value'], method='bonferroni')
cont_filt['p_adjusted_bonf'] = p_adjusted
cont_filt.head()

,Control 1,Control 2,Control 3,Control 4,Control 5,Control 6,Control 7,Control 8,Control 9,Control 10,...,ALS 25,ALS 26,ALS 27,ALS 28,ALS 29,U-stat,p_value,p_adjusted,p_adjusted_bh,p_adjusted_bonf
Gene names,,,,,,,,,,,,,,,,,,,,,
A1BG,26.95427,26.04318,26.86112,26.84817,26.46922,26.64471,26.84086,26.65403,27.17303,26.97836,...,26.83473,27.07398,26.92603,26.98934,26.40678,455.0,0.441766,0.920734,0.920734,1.00000
A2M,34.05310,33.97562,34.44289,34.01380,33.76643,33.97143,33.94585,33.94951,34.24133,34.00864,...,33.92804,33.91019,33.32133,34.02383,33.80540,582.0,0.004503,0.297230,0.297230,0.89169
ACTB,30.33841,29.20109,30.07712,30.10600,29.82487,29.17995,30.12866,29.74278,29.49278,29.82696,...,30.15943,30.55499,30.32978,30.44391,29.81420,362.0,0.490382,0.964701,0.964701,1.00000
ADIPOQ,29.63951,28.92731,30.02842,29.37902,30.37171,31.21824,29.76444,28.42606,29.57760,30.02842,...,30.76462,29.90928,28.78578,30.34604,31.70989,399.0,0.918017,0.964701,0.964701,1.00000
AFM,28.50805,28.99558,30.35100,28.39239,23.28411,28.72576,27.65581,28.57423,27.04895,26.97367,...,27.27897,26.52943,28.17736,27.89406,29.41353,460.5,0.395990,0.900817,0.900817,1.00000


In [15]:
#df['average'] = df.mean(numeric_only=True, axis=1)
cont_filt['Mean Control'] = cont_filt.loc[:, cont_filt.columns.str.startswith('Control')].mean(axis=1)
cont_filt['Mean ALS'] = cont_filt.loc[:, cont_filt.columns.str.startswith('ALS')].mean(axis=1)

In [16]:
#Fold Change
cont_filt["FC ALS-C"]= cont_filt["Mean ALS"]/cont_filt["Mean Control"]

In [17]:
#Log 2 FC
cont_filt["Log2FC ALS-C"]=np.log2(cont_filt["FC ALS-C"])
cont_filt.head()

,Control 1,Control 2,Control 3,Control 4,Control 5,Control 6,Control 7,Control 8,Control 9,Control 10,...,ALS 29,U-stat,p_value,p_adjusted,p_adjusted_bh,p_adjusted_bonf,Mean Control,Mean ALS,FC ALS-C,Log2FC ALS-C
Gene names,,,,,,,,,,,,,,,,,,,,,
A1BG,26.95427,26.04318,26.86112,26.84817,26.46922,26.64471,26.84086,26.65403,27.17303,26.97836,...,26.40678,455.0,0.441766,0.920734,0.920734,1.00000,26.729428,26.693026,0.998638,-0.001966
A2M,34.05310,33.97562,34.44289,34.01380,33.76643,33.97143,33.94585,33.94951,34.24133,34.00864,...,33.80540,582.0,0.004503,0.297230,0.297230,0.89169,34.005435,33.820613,0.994565,-0.007863
ACTB,30.33841,29.20109,30.07712,30.10600,29.82487,29.17995,30.12866,29.74278,29.49278,29.82696,...,29.81420,362.0,0.490382,0.964701,0.964701,1.00000,29.837516,29.924744,1.002923,0.004211
ADIPOQ,29.63951,28.92731,30.02842,29.37902,30.37171,31.21824,29.76444,28.42606,29.57760,30.02842,...,31.70989,399.0,0.918017,0.964701,0.964701,1.00000,30.069430,29.986205,0.997232,-0.003999
AFM,28.50805,28.99558,30.35100,28.39239,23.28411,28.72576,27.65581,28.57423,27.04895,26.97367,...,29.41353,460.5,0.395990,0.900817,0.900817,1.00000,28.017098,27.760570,0.990844,-0.013270


In [18]:
cont_filt.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Machine learning\ML stats Mann-W.xlsx")

In [19]:
# Sort the DataFrame by the absolute values of the column
down_sorted = cont_filt.loc[cont_filt["Log2FC ALS-C"].abs().sort_values(ascending=False).index]



In [20]:
down_sorted.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Machine learning\ML Mann-W sorted.xlsx")

In [21]:
# Filter rows where 'P-value' <= 0.05
filtered_down = down_sorted[down_sorted['p_value'] <= 0.05]

In [22]:
filtered_down.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Machine learning\ML Mann-W filtered.xlsx")